In [1]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
import time

from joblib import Parallel, delayed
from sklearn.compose import ColumnTransformer

from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler

from sklego.linear_model import DemographicParityClassifier, EqualOpportunityClassifier

tf.disable_eager_execution()
pd.set_option('display.max_columns', None)

## Loading FVC classes

In [2]:
from NotebookHelperFunctions import *
from Objectives import *
from FairVotingClassifier import *

<h2>Loading the dataset</h2>

In [3]:
n_runs = 30
seed_list = list(range(n_runs))
verbose = False

pop_size = 250
n_gen = 50
n_jobs = -1
max_iter = 10000

In [4]:
dataset = 'german'

df = pd.read_csv('datasets/german_credit_data.csv')
df['Job'] = df['Job'].replace({0: 'unskilled and non-resident', 1: 'unskilled and resident', 2: 'skilled', 3: 'highly skilled'})
df['Risk'] = df['Risk'].replace({'good': 0, 'bad': 1})
df.tail()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
995,31,female,unskilled and resident,own,little,NaN,1736,12,furniture/equipment,0
996,40,male,highly skilled,own,little,little,3857,30,car,0
997,38,male,skilled,own,little,NaN,804,12,radio/TV,0
998,23,male,skilled,free,little,little,1845,45,radio/TV,1
999,27,male,skilled,own,moderate,moderate,4576,45,car,0


<h3>Setting the protected attributes, unprivileged groups and the positive outcome</h3>

In [5]:
is_ohe = True
protected_attr = 'Sex'
unprivileged_values = ['female']
privileged_values = ['male']
best_outcome = np.array([0])
objective = 'Risk'

<h3>One-hot encoding</h3>

In [6]:
num_columns = ['Age', 'Credit amount', 'Duration']
cat_columns = ['Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']

df[num_columns] = MaxAbsScaler().fit_transform(df[num_columns])
df_dummies = pd.get_dummies(df[cat_columns + [protected_attr]])
cat_columns_dummies = df_dummies.columns.tolist()

df = pd.concat([df.drop(cat_columns, axis=1), df_dummies], axis=1)
#df = pd.concat([df, df_dummies], axis=1)
df.tail()

,Age,Sex,Credit amount,Duration,Risk,Job_highly skilled,Job_skilled,Job_unskilled and non-resident,Job_unskilled and resident,Housing_free,Housing_own,Housing_rent,Saving accounts_little,Saving accounts_moderate,Saving accounts_quite rich,Saving accounts_rich,Checking account_little,Checking account_moderate,Checking account_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Sex_female,Sex_male
995,0.413333,female,0.094225,0.166667,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
996,0.533333,male,0.209347,0.416667,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
997,0.506667,male,0.043639,0.166667,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
998,0.306667,male,0.100141,0.625000,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1
999,0.360000,male,0.248372,0.625000,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1


<h2>Running all algorithms</h2>

In [7]:
results = []
run_info = []

for seed in seed_list:
    display(f'Seed {seed}')
    np.random.seed(seed)

    classifiers = get_classifiers(seed, max_iter)
    all_classes = np.append(best_outcome, np.delete(np.unique(df[objective]), best_outcome))

    # Splitting the dataset
    X_train, X_test, y_train, y_test = train_test_split(df.drop(objective, axis=1), df[objective], stratify=df[objective], test_size=0.3, random_state=seed)

    # Separating the protected attribute to be used by other algorithms later
    X_protected_train = X_train[protected_attr]
    X_protected_test = X_test[protected_attr]

    # Removing the protected attribute before one-hot encoding
    X_train = X_train.drop(protected_attr, axis=1)
    X_test = X_test.drop(protected_attr, axis=1)
    
    # Simple Voting Ensemble
    display('Training the Voting Ensemble')
    voting = VotingClassifier(estimators=classifiers, voting='soft', n_jobs=n_jobs, verbose=verbose)
    voting = voting.fit(X_train, y_train)

    res, info = get_standard_classifier_results(voting, cat_columns_dummies, X_test, y_test, 'Voting', np.nan, all_classes, best_outcome, is_ohe,
                                                n_jobs, protected_attr, privileged_values, unprivileged_values, seed)
    results.extend(res)
    run_info.extend(info)

    # Voting + Fairlearn
    display('Training Fairlearn')
    res, info = get_fairlearn_results(voting, cat_columns_dummies, X_train, y_train, X_test, y_test, X_protected_train, X_protected_test,
                                      all_classes, best_outcome, is_ohe, n_jobs, protected_attr, privileged_values,
                                      unprivileged_values, seed)
    results.extend(res)
    run_info.extend(info)
    
    # Scikit-lego
    try:
        display('Training scikit-lego - DemographicParityClassifier')
        dpclf = DemographicParityClassifier(sensitive_cols=protected_attr, train_sensitive_cols=False, max_iter=max_iter,
                                            covariance_threshold=None, n_jobs=n_jobs)
        dpclf = dpclf.fit(X_train, y_train)
        res, info = get_standard_classifier_results(dpclf, cat_columns_dummies, X_test, y_test, 'scikit-lego', 'DemographicParityClassifier',
                                                    all_classes, best_outcome, is_ohe, n_jobs, protected_attr, privileged_values,
                                                    unprivileged_values, seed)
        results.extend(res)
        run_info.extend(info)
    except Exception as e:
        display(e)

    try:
        display('Training scikit-lego - EqualOpportunityClassifier')
        eoclf = EqualOpportunityClassifier(sensitive_cols=protected_attr, positive_target=best_outcome,
                                           train_sensitive_cols=False, max_iter=max_iter, covariance_threshold=None,
                                           n_jobs=n_jobs)
        eoclf = eoclf.fit(X_train, y_train)
        res, info = get_standard_classifier_results(eoclf, cat_columns_dummies, X_test, y_test, 'scikit-lego', 'EqualOpportunityClassifier',
                                                    all_classes, best_outcome, is_ohe, n_jobs, protected_attr,
                                                    privileged_values, unprivileged_values, seed)
        results.extend(res)
        run_info.extend(info)
    except Exception as e:
        display(e)
    
    # Proposal
    for engine in ['nsga2', 'rnsga2']:
        display(f'Training the proposal ({engine})')
        fairclf = FairVotingClassifier(estimators=classifiers, protected_attribute=protected_attr, engine=engine,
                                       unprivileged_values=unprivileged_values, positive_outcome=best_outcome,
                                       privileged_values=privileged_values, all_classes=y_train,
                                       verbose=verbose, is_protected_ohe=is_ohe, random_state=seed, n_jobs=n_jobs)
        fairclf.fit(X_train, y_train, pop_size=pop_size, n_gen=n_gen)

        technique = 'Proposal'
        algorithm = engine
        res, info = get_standard_classifier_results(fairclf, cat_columns_dummies, X_test, y_test, technique, algorithm, all_classes, 
                                                    best_outcome, is_ohe, n_jobs, protected_attr, privileged_values,
                                                    unprivileged_values, seed, weights=None)
        results.extend(res)
        run_info.extend(info)

        # getting the best, weighted by objective function
        technique = 'ProposalWeighted'
        for i in range(5):
            algorithm = f'F{i+1}_{engine}'
            topsis_weights = [0]*5
            topsis_weights[i] = 1
            _, weights = fairclf.rank_solutions(get_n_best=1, topsis_weights=topsis_weights, as_dataframe=False)
            res, info = get_standard_classifier_results(fairclf, cat_columns_dummies, X_test, y_test, technique, algorithm, all_classes, 
                                                        best_outcome, is_ohe, n_jobs, protected_attr, privileged_values,
                                                        unprivileged_values, seed, weights=weights.flatten())
            results.extend(res)
            run_info.extend(info)
    
    # AIF360
    display(f'Training AIF360 - AdversarialDebiasing')
    aifclf = AdversarialDebiasing(verbose=verbose, random_state=seed, num_epochs=n_gen)
    aifclf = aifclf.fit(convert_df_to_aif_inprocessing(X_train, cat_columns_dummies, protected_attr), y_train)

    res, info = get_standard_classifier_results(aifclf, cat_columns_dummies, X_test, y_test, 'AIF360', 'AdversarialDebiasing', all_classes,
                                                best_outcome, is_ohe, n_jobs, protected_attr, privileged_values,
                                                unprivileged_values, seed, is_aif=True)
    results.extend(res)
    run_info.extend(info)

    display(f'Training AIF360 - PostProcessingMeta')
    aifpostprocessing = PostProcessingMeta(voting, postprocessor=CalibratedEqualizedOdds(random_state=seed))
    X_train_aif = convert_df_to_aif_postprocessing(X_train, cat_columns_dummies, protected_attr, privileged_values, unprivileged_values)
    X_test_aif = convert_df_to_aif_postprocessing(X_test, cat_columns_dummies, protected_attr, privileged_values, unprivileged_values)
    y_train.index = X_train_aif.index
    y_test.index = X_test_aif.index
    aifpostprocessing.fit(X_train_aif, y_train)
    res, info = get_aif_postprocessing_classifier_results(aifpostprocessing, cat_columns_dummies, protected_attr, X_test_aif, y_test,
                                                          'AIF360', 'PostProcessingMeta', all_classes, best_outcome, False, n_jobs,
                                                          seed=seed, verbose=verbose)

    results.extend(res)
    run_info.extend(info)

'Seed 0'

'Training the Voting Ensemble'

'Training Fairlearn'

'Training scikit-lego - DemographicParityClassifier'

'Training scikit-lego - EqualOpportunityClassifier'

...

'Training scikit-lego - EqualOpportunityClassifier'

'Training the proposal (nsga2)'

'Training the proposal (rnsga2)'

'Training AIF360 - AdversarialDebiasing'

'Training AIF360 - PostProcessingMeta'

## Storing results

In [8]:
cols_objectives = ['F1', 'F2', 'F3', 'F4', 'F5']
df_results = pd.DataFrame(np.column_stack([run_info, results]),
                          columns=['Technique', 'Algorithm', 'Classifier', 'Seed'] + cols_objectives)
df_results = df_results.replace({'nan': np.nan})
for col in cols_objectives:
    df_results[col] = pd.to_numeric(df_results[col])
df_results.to_csv(f'outputs/df_results{dataset}.csv', sep='\t', encoding='utf-8')